* Currently, every time I want to summarize data, I re-run the entire query (for both the season-wide ridgeplot, and the week-by-week histogram).
* Re-running the queries over the entire data set takes a combined ~10 minutes.

That's a very long time. So, instead, let's make a summary table witin the database, and query it when we need to retrieve summaries. This will save a lot of time, because we don't need to recalculate it every time.

## I. spec table
We need the spec-level-count summary. 
* First, we will do an initial group by over the entire roster/run join table. This is what we do multiple times a day now to get the summary. This time, instead of piping the summary out, we will save it as the "summary_spec" table within MDB.

* Second, to update the summary table with new data, we will make a sql query that will:
    * Create a summary for the sub-set of the roster/run join table LIMITED TO THE CURRENT WEEK ONLY (and week CURRENT-1 to account for any sort of region-based period overlaps). Intead of taking 3-4 minutes, this should only take a few seconds.
    * Insert results of the CURRENT WEEK summary into the summary table. Within the CURERNT_WEEK subset, we'll just recalculate/overwrite the relevant rows in the summary table every time the query is called.

In [1]:
import pandas as pd
import mplusdb

In [4]:
mdb = mplusdb.MplusDatabase('.db_config')

### spec table

In [17]:
# Create the initial summary table -- we'll have to pipe thse data outside first, bc
# the buffer size is too small to handle this operation with a CREATE SELECT

# run this directly in the mdb
CREATE_EMPTY_TABLE = """
CREATE TABLE summary_spec (
    period int unsigned default 0 not null,
    spec int unsigned default 0 not null,
    level int unsigned default 0 not null,
    count int unsigned default 0 not null,
    PRIMARY KEY (period, spec, level)
);
"""

# summarize all of the available data
SUMMARY_QUERY = """
SELECT period, spec, level, count(level) as count
FROM new_table
INNER JOIN roster
ON new_table.id = roster.run_id
GROUP BY period, spec, level;
"""

#insert the summary data into the summary table
INSERT_QUERY = """
INSERT INTO summary_spec values(%s, %s, %s, %s)
"""
conn = None
data = None
try:
    conn = mdb.connect()
    cursor = conn.cursor()
    cursor.execute(SUMMARY_QUERY)
    data = cursor.fetchall()
    cursor.executemany(INSERT_QUERY, data)
    conn.commit()
except:
    print("an oopsie, closing connection")
finally:
    cursor.close()
    conn.close()


In [99]:
# update the spec summary with something like this
# (this will go into the summarize.py script)
UPDATE_QUERY = """
INSERT INTO summary_spec(period, spec, level, count)
SELECT period, spec, level, count(level) as count
FROM new_table
INNER JOIN roster on new_table.id = roster.run_id
WHERE new_table.period BETWEEN 770 AND 772
GROUP BY period, spec, level
ON DUPLICATE KEY UPDATE count=VALUES(count);
"""

In [100]:
import importlib
import summarize
import pandas as pd

In [101]:
importlib.reload(summarize)
data = summarize.get_runs_summary()

In [102]:
data

,season,spec,level,count
0,bfa4,62,2,74334
1,bfa4,62,3,21381
2,bfa4,62,4,35278
3,bfa4,62,5,53770
4,bfa4,62,6,50560
...,...,...,...,...
1876,bfa4_postpatch,581,21,229
1877,bfa4_postpatch,581,22,135
1878,bfa4_postpatch,581,23,83
1879,bfa4_postpatch,581,24,28


In [85]:
summarize.push_runs_summary_to_sqlite(data)